In [1]:
import json
import numpy as np
from sklearn.metrics.pairwise import distance_metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import urllib2
import csv

In [2]:
# 'https://www.airbnb.com/rooms/825531?s=vJx50hcn'

headers = {"User-agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"}
search = input('Enter url: ')
search_id = search[search.find('rooms/')+6:search.find('?')]
url = 'https://api.airbnb.com/v2/listings/' + search_id + '?client_id=3092nxybyb0otqw18e8nh5nty&_format=v1_legacy_for_p3'
request = urllib2.Request(url, headers=headers)
data = json.loads(urllib2.urlopen(request).read())
description = data['listing']['description']
print(description)

Enter url: 'https://www.airbnb.com/rooms/825531?s=vJx50hcn'
Apartment is located in the residential neighborhood of Roscoe Village. There are restaurants and bars within a 10 minute walk, but you can easily hop on one of the nearby bus lines to reach any other neighborhood in the city. 

Furnished one bedroom apartment available for short term rent.  Free street parking and convenient to public transit. (Damen, Belmont, Diversey and Clybourn buses).

Bedroom features a full-size bed with fresh linens. Additional sleeping areas available on living room futon.

Wifi and basic cable. Fully equipped kitchen with microwave and AC for the Summer months. 

Located in Roscoe Village, a safe and active neighborhood easily accessible to Wrigley Field, Downtown, and popular bars and restaurants. Awesome taco joint open till 3 AM is my favorite go-to after a night out.

Please help yourself to toiletries in the bathroom and anything in the kitchen that you need for a comfortable stay.

I travel a 

In [5]:
#make a dictionary of descriptions: descriptions[listing_id] = description
#make a tf-idf matrix of words by listings
#cosine similarity

#load the data
with open('data/filtered_nyc_listings.csv') as f:
     NYClistings = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]
        
with open('data/filtered_sf_listings.csv') as f:
     SFlistings = [{k: v for k, v in row.items()}
          for row in csv.DictReader(f, skipinitialspace=True)]
        
descript_dict = {}
for n in NYClistings:
    key = n['id']
    descript_dict[key]=n['description']
    
for s in SFlistings:
    key = s['id']
    descript_dict[key]=s['description']
    


In [6]:
n_feats = 5000
descriptions = [descript_dict[d] for d in descript_dict]
print(descriptions[0])
listing_by_vocab = np.empty((len(descriptions), n_feats))


tfidf_vec = TfidfVectorizer(min_df = 10, max_df = .8, max_features = n_feats, norm='l2', stop_words = 'english')
listing_by_vocab = tfidf_vec.fit_transform(descriptions).toarray()


Welcome to the HART House! A safe and clean place for travelers and artists to sleep, rest, create, and get to know Spanish Harlem. We have one spacious and sunny dorm room available for travelers, it accommodates up to 4 people. The HART house is a gorgeous 5 bedroom, 2 bathroom apartment that is rented and run by the Harlem Art Collective. The apartment takes up the entire 5th floor of our building and is full of light, art, and creativity. Our walls are covered with murals and art from local artists and members of our collective, and we are always available to show you around the neighborhood and the art scene!  Our apartment is also a unique creative space that we use for art workshops, artist studio space, and Harlem Art Collective meetings and events, as well as a back packers haven and travelers hang out! Our guests are welcome and encouraged to tap into their creative side and paint or create art while staying with us!  The proceeds from your stay go directly into the the Harle

In [ ]:
# get input description in a tf-idf vector
# compute the cosine similarity between each of the other descriptions
# sort the results